In [1]:
import numpy as np

In [2]:
def nonempty_count(M):
    count = 0
    for i in range(M.shape[0]):
        for j in range(M.shape[1]):
            if M[i, j] == -1:
                count += 1
    return M.size - count


def init_u_v(M, d):
    count = 0
    sum = 0
    
    for i in range(M.shape[0]):
        for j in range(M.shape[1]):
            
            if not M[i, j] == -1:
                sum += M[i,j]
                count += 1
         
    avg = np.sqrt((sum / count) / d)

    m, n = M.shape
    
    u = np.ones([m, d]) * avg
    v = np.ones([d, n]) * avg
    return u, v
    

In [3]:
def RMSE(T, M):
    
#     T = np.dot(u, v)
    
    error = 0
    for j in range(M.shape[1]):
        
        row_error = 0
        for i in range(M.shape[0]):
            if M[i][j] == -1:
                continue
            row_error += (M[i, j] - T[i, j])**2
            
        error += row_error
        
    return np.sqrt(error / nonempty_count(M)) 

In [4]:
def solve_U_at(r, s, u, v, M):
    denom = 0 
    numer = 0
    
    for j in range(v.shape[1]):
        
        if M[r, j] == -1:
            continue

        if v[s, j] != -1:
            denom += v[s, j]**2
    
    
    for j in range(v.shape[1]):
        if M[r, j] == -1:
            continue
        
        sum1 = 0
        for k in range(u.shape[1]):
            if k == s:
                continue
            sum1 += u[r, k] * v[k, j]
    
        numer += v[s, j] * (M[r, j] - sum1)
        
    
    return numer/denom
    
    

In [5]:
def solve_V_at(r, s, u, v, M):
    numer = 0
    denom = 0
    
    for i in range(u.shape[0]):
        if M[i, s] == -1:
            continue            
        if u[i,r] != -1:
            denom += u[i, r]**2
            
    
    for i in range(u.shape[0]):
        if M[i, s] == -1:
            continue            
        
        sum1 = 0
        for k in range(v.shape[0]):
            if k == r:
                continue
            sum1 += u[i, k] * v[k, s]
        
        numer += u[i, r] * (M[i, s] - sum1)
        
    return numer/denom
    

In [6]:
def solve_U(u, v, M):
    for r in range(u.shape[0]):
        for s in range(u.shape[1]):
            u[r, s] = solve_U_at(r, s, u, v, M)
            
    return u

def solve_V(u, v, M):
    for r in range(v.shape[0]):
        for s in range(v.shape[1]):
            v[r, s] = solve_V_at(r, s, u, v, M)
            
    return v



In [32]:
M = np.array([
    [2, 1, -1, 1, -1, 5],
    [4, -1, 2, -1, -1, -1], 
    [3, 3, -1, 5, 1, -1], 
    [-1, -1, 5, -1, 1, 2], 
])

u, v = init_u_v(M, 3)
print(RMSE(np.dot(u, v), M))

for i in range(1000):
    u = solve_U(u, v, M)
    v = solve_V(u, v, M)
    
M_hat = np.dot(u, v)
print(RMSE(M_hat, M))

1.53846153846
1.4442750009e-16


In [33]:
print(u)

[[ 0.35431061  1.95145483 -0.09459433]
 [ 0.04725739  1.14036959  1.72440795]
 [ 1.36575654  0.70399184  1.19569369]
 [ 2.02043111  0.55626763  1.75488665]]


In [34]:
print(v)

[[ 0.23881216  0.98529201  1.61340011  2.54010604 -0.4794647   0.52538734]
 [ 1.05967228  0.3894971   0.35998014  0.11016697  0.96449231  2.45500175]
 [ 1.61231936  1.15424897  0.87754439  1.21542575  0.81612652 -0.24340428]]


In [39]:
print(M_hat)

[[ 2.          1.          1.19111902  1.          1.63508281  5.        ]
 [ 4.          2.48112908  2.          2.34155968  2.48455451  2.40470948]
 [ 3.          3.          3.50620912  5.          1.          2.15481543]
 [ 3.90140264  4.23295536  5.          7.32632601  1.          2.        ]]


In [36]:
print(M)

[[ 2  1 -1  1 -1  5]
 [ 4 -1  2 -1 -1 -1]
 [ 3  3 -1  5  1 -1]
 [-1 -1  5 -1  1  2]]
